In [ ]:
!pip install pyautogen
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

config_list = {
        "model": "llama3-70b-8192",
        "api_key": userdata.get('GROQ_API_KEY'),
        "api_type": "groq",
}

In [ ]:
import autogen

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message="You are a user who provides descriptions of cloud system architectures, flow charts, mind maps etc. that need to be converted into d2 code language.",
    code_execution_config=False,
    human_input_mode="NEVER"
)


diagram_generator = autogen.AssistantAgent(
    name="Diagram_Generator",
    system_message="""You are an assistant who generates Python code for creating diagrams based on the user's description of cloud system architectures. You use the Diagrams library to create these diagram code. Ensure your code is complete and executable without requiring modification.
    Provide the full Python code wrapped in a code block for execution. If there are errors, correct them and provide the updated code. If the task cannot be solved even after successful execution, analyze the problem, revisit your assumptions, gather additional information, and think of a different approach to try.
    Refer to the following examples to understand the format and style:

Example 1:
User: "Create a diagram with a load balancer and five EC2 worker instances connected to an RDS database."
Code:
python
from diagrams import Diagram
from diagrams.aws.compute import EC2
from diagrams.aws.database import RDS
from diagrams.aws.network import ELB

with Diagram("Grouped Workers", show=False, direction="TB"):
    ELB("lb") >> [EC2("worker1"),
                  EC2("worker2"),
                  EC2("worker3"),
                  EC2("worker4"),
                  EC2("worker5")] >> RDS("events")

Example 2:
User: "Create a clustered web services architecture with Route 53 DNS, an ELB, three ECS services, an RDS database cluster with a primary and a read replica, and an ElastiCache for memcached."
Code:
python
from diagrams import Cluster, Diagram
from diagrams.aws.compute import ECS
from diagrams.aws.database import ElastiCache, RDS
from diagrams.aws.network import ELB
from diagrams.aws.network import Route53

with Diagram("Clustered Web Services", show=False):
    dns = Route53("dns")
    lb = ELB("lb")

    with Cluster("Services"):
        svc_group = [ECS("web1"),
                     ECS("web2"),
                     ECS("web3")]

    with Cluster("DB Cluster"):
        db_primary = RDS("userdb")
        db_primary - [RDS("userdb ro")]

    memcached = ElastiCache("memcached")

    dns >> lb >> svc_group
    svc_group >> db_primary
    svc_group >> memcached


Example 3:
User: "Create an event processing architecture with an EKS source, ECS workers, an SQS event queue, Lambda functions for processing, and storage in S3 and Redshift."
Code:

from diagrams import Cluster, Diagram
from diagrams.aws.compute import ECS, EKS, Lambda
from diagrams.aws.database import Redshift
from diagrams.aws.integration import SQS
from diagrams.aws.storage import S3

with Diagram("Event Processing", show=False):
    source = EKS("k8s source")

    with Cluster("Event Flows"):
        with Cluster("Event Workers"):
            workers = [ECS("worker1"),
                       ECS("worker2"),
                       ECS("worker3")]

        queue = SQS("event queue")

        with Cluster("Processing"):
            handlers = [Lambda("proc1"),
                        Lambda("proc2"),
                        Lambda("proc3")]

    store = S3("events store")
    dw = Redshift("analytics")

    source >> workers >> queue >> handlers
    handlers >> store
    handlers >> dw


Example 4:
User: "Create a message collecting system on GCP with IoT Core sources, PubSub for messaging, Dataflow for data processing, and targets including BigQuery, GCS, AppEngine, and BigTable."
Code:
from diagrams import Cluster, Diagram
from diagrams.gcp.analytics import BigQuery, Dataflow, PubSub
from diagrams.gcp.compute import AppEngine, Functions
from diagrams.gcp.database import BigTable
from diagrams.gcp.iot import IotCore
from diagrams.gcp.storage import GCS

with Diagram("Message Collecting", show=False):
    pubsub = PubSub("pubsub")

    with Cluster("Source of Data"):
        [IotCore("core1"),
         IotCore("core2"),
         IotCore("core3")] >> pubsub

    with Cluster("Targets"):
        with Cluster("Data Flow"):
            flow = Dataflow("data flow")

        with Cluster("Data Lake"):
            flow >> [BigQuery("bq"),
                     GCS("storage")]

        with Cluster("Event Driven"):
            with Cluster("Processing"):
                flow >> AppEngine("engine") >> BigTable("bigtable")

            with Cluster("Serverless"):
                flow >> Functions("func") >> AppEngine("appengine")

    pubsub >> flow
    """,
    llm_config=config_list
)

code_validator = autogen.AssistantAgent(
    name="Code_Validator",
    system_message="""You are a validator who reviews the Diagrams Python code generated for creating diagrams. Your task is to check the code for correctness, completeness, and adherence to the Diagrams library standards.
    If you find any issues or errors, provide detailed feedback and suggestions for correction. If the code is correct, approve it.
    """,
    llm_config=config_list
)


critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""You are a critic who double-checks the plan, claims, and code from other agents. Provide feedback and suggest improvements to ensure the highest quality of the generated diagrams and the overall process.
    """,
    llm_config=config_list
)

groupchat = autogen.GroupChat(agents=[user_proxy, diagram_generator, code_validator, critic], messages=[], max_round=8)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=config_list)

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="Create a data processing pipeline diagram with a data source, an ETL process, a data warehouse, and a business intelligence tool."
)

User (to chat_manager):

Create a data processing pipeline diagram with a data source, an ETL process, a data warehouse, and a business intelligence tool.

--------------------------------------------------------------------------------

Next speaker: Diagram_Generator

Diagram_Generator (to chat_manager):

Here is the Python code to create a data processing pipeline diagram with a data source, an ETL process, a data warehouse, and a business intelligence tool using the Diagrams library:
```
from diagrams import Diagram, Cluster
from diagrams.aws.storage import S3
from diagrams.programming.framework import ApacheAirflow
from diagrams.aws.database import Redshift
from diagrams.onpremise.informational import Tableau

with Diagram("Data Processing Pipeline", direction="LR", show=False):
    source = S3("Data Source")

    with Cluster("ETL Process"):
        etl = ApacheAirflow("ETL")

    dw = Redshift("Data Warehouse")

    bi = Tableau("Business Intelligence")

    source >> etl >> dw 

ChatResult(chat_id=None, chat_history=[{'content': 'Create a data processing pipeline diagram with a data source, an ETL process, a data warehouse, and a business intelligence tool.', 'role': 'assistant'}, {'content': 'Here is the Python code to create a data processing pipeline diagram with a data source, an ETL process, a data warehouse, and a business intelligence tool using the Diagrams library:\n```\nfrom diagrams import Diagram, Cluster\nfrom diagrams.aws.storage import S3\nfrom diagrams.programming.framework import ApacheAirflow\nfrom diagrams.aws.database import Redshift\nfrom diagrams.onpremise.informational import Tableau\n\nwith Diagram("Data Processing Pipeline", direction="LR", show=False):\n    source = S3("Data Source")\n\n    with Cluster("ETL Process"):\n        etl = ApacheAirflow("ETL")\n\n    dw = Redshift("Data Warehouse")\n\n    bi = Tableau("Business Intelligence")\n\n    source >> etl >> dw >> bi\n```\nThis code generates a diagram with the following components:

In [ ]:
!pip install diagrams

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.7/824.7 kB 45.2 MB/s eta 0:00:00


In [ ]:
! pip install "pyautogen>=0.2.18" "tavily-python"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 923.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
